In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from downhole.survey import Survey, DownholeSurvey, Hole, Sample, Sample_Set, Hole_Set
from downhole.examples.ex_survey import *

## Import the Excel Sheets into Pandas Dataframes

In [3]:
collar_df = pd.read_excel('.\\Test Spreadsheets\\Collar.xlsx', index_col=0)
survey_df = pd.read_excel('.\\Test Spreadsheets\\Survey.xlsx', index_col=0)
geochem_df = pd.read_excel('.\\Test Spreadsheets\\CR_Static_Geochem.xlsx',sheet_name=1, index_col=1)

## Initialize the Hole Set

In [4]:
hs = Hole_Set() # Hole_Set object to retain the list of holes in the collar table
for hole in list(pd.unique(collar_df.index)): # list of hole id's in the collar table
    hs.add_hole(hole)

print(len(hs.holes)) # print the number of holes imported

522


## Initialize the Survey Set

In [5]:
s = Survey()
for hole in hs.holes:
    try:
        if len(survey_df.loc[[hole]]) == 1:
            print(f"No Downhole Survey {hole}, collar info used.")
            ds = DownholeSurvey(
                [collar_df.at[hole, 'Azimuth_GridN'],collar_df.at[hole, 'Azimuth_GridN']],
                [collar_df.at[hole, 'DIP'],collar_df.at[hole, 'DIP']],
                [0,collar_df.at[hole, 'TotalDepth_m']],
                collar_df.at[hole, 'UTM_N83_EASTING'],
                collar_df.at[hole, 'UTM_N83_NORTHING'],
                collar_df.at[hole, 'UTM_N83_ELEV_MSL'], hole
            )
        else:
            azm_data = list(survey_df.loc[[hole]]['AzimGridN'])
            dip_data = list(survey_df.loc[[hole]]['Final_Dip'])
            depth_data = list(survey_df.loc[[hole]]['Depth_m'])
            
            if collar_df.at[hole, 'TotalDepth_m'] > depth_data[-1]:
                difference = collar_df.at[hole, 'TotalDepth_m'] - depth_data[-1]
                print(f"Collar depth > survey depth by {collar_df.at[hole, 'TotalDepth_m']-depth_data[-1]} for {hole}, interval added. {difference}")
                azm_data.insert(len(azm_data),azm_data[-1])
                dip_data.insert(len(dip_data),dip_data[-1])
                depth_data.insert(len(depth_data),collar_df.at[hole, 'TotalDepth_m'])
            
            if min(survey_df.loc[[hole]]['Depth_m']) > 3:
                print(f"{hole} No survey reading for first {min(survey_df.loc[[hole]]['Depth_m'])} m, interval added from collar table.")
                azm_data.insert(0,collar_df.at[hole, 'Azimuth_GridN'])
                dip_data.insert(0,collar_df.at[hole, 'DIP'])
                depth_data.insert(0,0.00)
                
            elif 3 >= min(survey_df.loc[[hole]]['Depth_m']) > 0:
                print(f"{hole} No survey reading for first {min(survey_df.loc[[hole]]['Depth_m'])} m, interval added from first survey interval.")
                azm_data.insert(0,azm_data[0])
                dip_data.insert(0,dip_data[0])
                depth_data.insert(0,0.00)
                
                
            ds = DownholeSurvey(
                azm_data,
                dip_data,
                depth_data,
                collar_df.at[hole, 'UTM_N83_EASTING'],
                collar_df.at[hole, 'UTM_N83_NORTHING'],
                collar_df.at[hole, 'UTM_N83_ELEV_MSL'], hole
            )
        

    except KeyError:
        print(f"No Downhole Survey or record of {hole}, collar info used.")
        ds = DownholeSurvey(
            [collar_df.at[hole, 'Azimuth_GridN'],collar_df.at[hole, 'Azimuth_GridN']],
            [collar_df.at[hole, 'DIP'],collar_df.at[hole, 'DIP']],
            [0,collar_df.at[hole, 'TotalDepth_m']],
            collar_df.at[hole, 'UTM_N83_EASTING'],
            collar_df.at[hole, 'UTM_N83_NORTHING'],
            collar_df.at[hole, 'UTM_N83_ELEV_MSL'], hole
        )
    
    s.add_downhole_survey(ds)


No Downhole Survey KDI-12-001, collar info used.
No Downhole Survey KDI-12-002, collar info used.
Collar depth > survey depth by 10.0 for KDI-12-005, interval added. 10.0
Collar depth > survey depth by 4.0 for KDI-12-007, interval added. 4.0
Collar depth > survey depth by 10.0 for KDI-12-008, interval added. 10.0
Collar depth > survey depth by 5.0 for KDI-12-009, interval added. 5.0
Collar depth > survey depth by 7.0 for KDI-12-010, interval added. 7.0
Collar depth > survey depth by 5.0 for KDI-12-011, interval added. 5.0
Collar depth > survey depth by 5.0 for KDI-12-012, interval added. 5.0
Collar depth > survey depth by 5.0 for KDI-12-013, interval added. 5.0
Collar depth > survey depth by 6.0 for KDI-12-014, interval added. 6.0
Collar depth > survey depth by 5.0 for KDI-12-015, interval added. 5.0
Collar depth > survey depth by 6.0 for KDI-13-001, interval added. 6.0
Collar depth > survey depth by 9.0 for KDI-13-002, interval added. 9.0
Collar depth > survey depth by 8.0 for KDI-13-

In [7]:
print(len(s.downhole_surveys), "downhole surveys.")

522 downhole surveys.


## Initialize the Sample Set

In [8]:
## Filter out samples with NULL FROM and TO values
cols = ['SAMPLE_TO_M', 'SAMPLE_FROM_M']
samples = list(geochem_df[geochem_df[cols].notna().all(1)][['HOLEID','SAMPLE_FROM_M','SAMPLE_TO_M']].reset_index().values)
len(samples)

122

In [38]:
ss = Sample_Set()
for downhole_survey in s.downhole_surveys:
    for sample in samples:
        if downhole_survey == sample[1]:
            sample_location = downhole_survey.min_curv.sample_interval(sample[2],sample[3])
            sample_object = Sample(sample[0], sample[1], 'INT', sample_location)
            ss.add_sample(sample_object)


In [40]:
len(ss.samples)
ss.export_shapefile()

In [32]:
s.downhole_surveys[0].min_curv.xyz_points()

[[597464.52, 7043372.34, 413.96], [597295.915, 7043427.774, 236.476]]

In [ ]:
s.export_shapefile()

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from math import sqrt

In [ ]:
intervals = s.downhole_surveys[40].min_curv.survey_intervals
intervals

In [ ]:
s.downhole_surveys[25].min_curv.sample_interval(5,25)

In [ ]:
s.downhole_surveys[25].min_curv.sample_point(2)
xs = [i[1][0] for i in intervals]
ys = [i[1][1] for i in intervals]
zs = [i[1][2] for i in intervals]

xs1 = [i[1][0] for i in s.downhole_surveys[40].min_curv.sample_interval(100,156)]
ys1 = [i[1][1] for i in s.downhole_surveys[40].min_curv.sample_interval(100,156)]
zs1 = [i[1][2] for i in s.downhole_surveys[40].min_curv.sample_interval(100,156)]


#x = [s.downhole_surveys[25].min_curv.sample_point(i)[1][0] for i in range(0,200,10)]
#y = [s.downhole_surveys[25].min_curv.sample_point(i)[1][1] for i in range(0,200,10)]
#z = [s.downhole_surveys[25].min_curv.sample_point(i)[1][2] for i in range(0,200,10)]

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.plot(xs,ys,zs, linewidth=1)
ax.plot(xs1,ys1,zs1, linewidth=4)

#ax.scatter(x,y,z)


In [ ]:
s.downhole_surveys[15].min_curv.sample_point(223)

In [ ]:
for downhole_survey in s.downhole_surveys:
    if str(downhole_survey) == 'KDI-12-001':
        print(downhole_survey.min_curv.sample_interval(5,10))
    